In [10]:
import os
import sys
import traceback
import random

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

In [11]:
 def tail(f, lines=20):
    total_lines_wanted = lines

    BLOCK_SIZE = 1024
    f.seek(0, 2)
    block_end_byte = f.tell()
    lines_to_go = total_lines_wanted
    block_number = -1
    blocks = []
    while lines_to_go > 0 and block_end_byte > 0:
        if (block_end_byte - BLOCK_SIZE > 0):
            f.seek(block_number*BLOCK_SIZE, 2)
            blocks.append(f.read(BLOCK_SIZE))
        else:
            f.seek(0,0)
            blocks.append(f.read(block_end_byte))
        lines_found = blocks[-1].count(b'\n')
        lines_to_go -= lines_found
        block_end_byte -= BLOCK_SIZE
        block_number -= 1
    all_read_text = b''.join(reversed(blocks))
    return b'\n'.join(all_read_text.splitlines()[-total_lines_wanted:]).decode("utf-8") 

In [12]:
def get_entries(dirname):
    os.chdir(dirname)
    data = {}
    for entry in os.listdir():
        if entry.startswith("a100"):
            entry_sp = entry.split("_")
            (gpu, job, topo, nnode_s, bw_s, deg_s, nlat_s, batch_s, njobs_s, nrun_s) = entry_sp
#             print(entry_sp)
            for innerdir in os.listdir(entry):
                if innerdir.startswith("nwsim"):
                    with open(entry+"/"+innerdir, "rb") as nw:
                        try:
                            tailn = tail(nw, int(njobs_s))
                            nums = [int(line.split(" ")[1]) for line in tailn.split("\n") if "Final" in line]
#                             print(nums)
                            if (int(bw_s), int(njobs_s)) in data:
                                data[int(bw_s), int(njobs_s)].extend(nums)
                            else:
                                data[int(bw_s), int(njobs_s)] = nums
                        except:
                            traceback.print_exception(*sys.exc_info())
    for k,v in data.items():
        data[k] = sorted(v)
    return data

In [13]:
data = get_entries("/home/gridsan/weiyangw/multijobs/fattree/job_load_abssw")

In [14]:
data

{(100, 15): [61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  61155050349,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  63968900177,
  210403519839,
  210403519839,
  210403519839,
  512880309661,
  512880309661,
  512880309661,
  512880309661,
  512880309661,
  5128

In [ ]:
os.chdir("/home/gridsan/weiyangw/multijobs/fattree/job_load_abssw")
for k, v in data.items():
    with open("{:d}_fattree_{:d}jobs.csv".format(k[0], k[1]), "w") as f:
        for i in v:
            f.write(str(i)+"\n")
    with open("{:d}_fattree_{:d}jobs_stat.csv".format(k[0], k[1]), "w") as f:
        avg_lat = sum(v)/len(v)
        tail_lat_50 = v[int(0.50*len(v)) if int(0.90*len(v)) < len(v) else -1]
        tail_lat_90 = v[int(0.90*len(v)) if int(0.90*len(v)) < len(v) else -1]
        tail_lat_95 = v[int(0.95*len(v)) if int(0.95*len(v)) < len(v) else -1]
        tail_lat_99 = v[int(0.99*len(v)) if int(0.99*len(v)) < len(v) else -1]
        f.write(str(avg_lat)+"\n")
        f.write(str(tail_lat_90)+"\n")
        f.write(str(tail_lat_95)+"\n")
        f.write(str(tail_lat_99)+"\n")
            

In [16]:
data = get_entries("/home/gridsan/weiyangw/multijobs/fattree/job_load_ft_32")

In [64]:
def get_topoopt_data(bw, deg):
    jobtype = ["dlrm", "transformer", "candle", "vggr"]
    chances = [0.35, 0.65, 0.9, 1]
    os.chdir("/home/gridsan/weiyangw/multijobs/topoopt")
    njobs = [5, 10, 15, 20, 27]
    data = {}
    for n in njobs:
        data[n] = []
        for trials in range(5):
            for m in range(n):
                jobname = jobtype[next(i for i, x in enumerate(chances) if x > random.uniform(0, 1))]
                data[n].append(sample_one(jobname, 16, bw, deg))
    for k,v in data.items():
        data[k] = sorted(v)
    return data

In [65]:
def sample_one(jname, nnode, bw, deg):
    avail = list(range(5))
    entries = get_entries_topoopt("/home/gridsan/weiyangw/multijobs/topoopt/"+jname+str(nnode))
    result = []
    while len(result) == 0:
        selrun = random.randint(0,4)
#         print(selrun)
        selected = filter(lambda e: int(e[4]) == bw and int(e[5]) == deg and int(e[-2]) == selrun, entries)
        result = list(selected)
#         print(result)
    return result[0][-1]

In [66]:
def get_entries_topoopt(dirname):
    os.chdir(dirname)
    data = []
    for entry in os.listdir():
        if entry.startswith("a100"):
            entry_sp = entry.split("_")
            last_line = ""
            for innerdir in os.listdir(entry):
                if innerdir.startswith("nwsim"):
                    with open(entry+"/"+innerdir, "rb") as nw:
                        try:
                            nw.seek(-2, os.SEEK_END)
                            while nw.read(1) != b'\n':
                                nw.seek(-2, os.SEEK_CUR)
                            last_line = nw.readline().decode()
                            if "Final" in last_line:
                                entry_sp.append(int(last_line.split(" ")[1][:-1]))
                                data.append(entry_sp)
                        except:
                            pass
    return data

In [67]:
get_entries_topoopt("/home/gridsan/weiyangw/multijobs/topoopt/vggr16")
sample_one("candle", 16, 100, 8)


131546054758

In [74]:
os.chdir("/home/gridsan/weiyangw/multijobs/topoopt")
topo_d8_16 = get_topoopt_data(100, 8)
print(topo_d8_16)
os.chdir("/home/gridsan/weiyangw/multijobs/topoopt")
for k, v in topo_d8_16.items():
    with open("{:d}_topo_d8_{:d}jobs.csv".format(100, k), "w") as f:
        for i in v:
            f.write(str(i)+"\n")
    with open("{:d}_topo_d8_{:d}jobs_stat.csv".format(100, k), "w") as f:
        avg_lat = sum(v)/len(v)
        tail_lat_50 = v[int(0.50*len(v)) if int(0.90*len(v)) < len(v) else -1]
        tail_lat_90 = v[int(0.90*len(v)) if int(0.90*len(v)) < len(v) else -1]
        tail_lat_95 = v[int(0.95*len(v)) if int(0.95*len(v)) < len(v) else -1]
        tail_lat_99 = v[int(0.99*len(v)) if int(0.99*len(v)) < len(v) else -1]
        f.write(str(avg_lat)+"\n")
        f.write(str(tail_lat_90)+"\n")
        f.write(str(tail_lat_95)+"\n")
        f.write(str(tail_lat_99)+"\n")

{5: [15185241946, 15185241946, 15185241946, 19926218536, 19926218536, 19926218536, 19926218536, 20917179248, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 131546054758, 131546054758], 10: [15185241946, 19382130971, 19382130971, 19611805047, 19611805047, 19611805047, 19611805047, 19926218536, 20917179248, 20917179248, 20917179248, 20917179248, 20917179248, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 62643560175, 100559060382, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758, 131546054758],

In [27]:
topo_d4_16 = get_topoopt_data(100, 4)
os.chdir("/home/gridsan/weiyangw/multijobs/topoopt")
for k, v in topo_d4_16.items():
    with open("{:d}_topo_d4_{:d}jobs.csv".format(100, k), "w") as f:
        for i in v:
            f.write(str(i)+"\n")
    with open("{:d}_topo_d4_{:d}jobs_stat.csv".format(100, k), "w") as f:
        avg_lat = sum(v)/len(v)
        tail_lat_90 = v[int(0.90*len(v)) if int(0.90*len(v)) < len(v) else -1]
        tail_lat_95 = v[int(0.95*len(v)) if int(0.95*len(v)) < len(v) else -1]
        tail_lat_99 = v[int(0.99*len(v)) if int(0.99*len(v)) < len(v) else -1]
        f.write(str(avg_lat)+"\n")
        f.write(str(tail_lat_90)+"\n")
        f.write(str(tail_lat_95)+"\n")
        f.write(str(tail_lat_99)+"\n")